In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import dns
import datetime
from time import sleep
import random
import json

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# Project defined dependencies
import config
from pprint import pprint
import cl_parser


In [30]:
# Create the craiglist parser
my_parser = cl_parser.Parser() 

In [31]:
for listing in my_parser.listings_collection.find():
    print(listing['data_id'])
    break

7278244211


In [32]:
# Get the current count of records
my_parser.listings_collection.find().count() 

7979

In [33]:
find_result = my_parser.listings_collection.find_one({'data_id': '7280736862'})
pprint(find_result)

{'_id': ObjectId('60330e42566e11a3b7aedc54'),
 'created_datetime': datetime.datetime(2021, 2, 22, 1, 52, 2, 943000),
 'data_id': '7280736862',
 'listing_addrcountry': 'US',
 'listing_address': '2051 Geneva St',
 'listing_addrlocality': 'Vista',
 'listing_addrregion': 'CA',
 'listing_addrstreet': '740 Paseo Buena Vista',
 'listing_addrzip': '92085',
 'listing_attributes': ['EV charging',
                        'cats are OK - purrr',
                        'dogs are OK - wooof',
                        'apartment',
                        'w/d in unit',
                        'no smoking',
                        'detached garage'],
 'listing_availability': 'available now',
 'listing_bath': 1,
 'listing_bed': '1',
 'listing_bedbath': '1BR / 1Ba',
 'listing_date': '2021-02-21 11:42',
 'listing_datetime': '2021-02-21 11:42',
 'listing_latitude': '33.208926',
 'listing_link': 'https://sandiego.craigslist.org/nsd/apa/d/vista-strike-while-the-iron-is-hot/7280736862.html',
 'listing_longitu

In [34]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Ben Galde\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [35]:
urls = []
urls.append(config.url_listings_northsd)
urls.append(config.url_listings_eastsd)
urls.append(config.url_lisings_cityofsd)
urls.append(config.url_lisings_southsd)
print(urls)

['https://sandiego.craigslist.org/search/nsd/apa', 'https://sandiego.craigslist.org/search/esd/apa', 'https://sandiego.craigslist.org/search/csd/apa', 'https://sandiego.craigslist.org/search/ssd/apa']


In [36]:
# saved page one of results locally for temp solution to build out scripts/functions
if config.test:
    filepath = os.path.join('..','resources','cl_sandiego_northcounty_apa.html')
    with open(filepath, encoding='utf-8') as file:
        html = file.read()
        soup = BeautifulSoup(html)
        results = soup.find_all('li', class_='result-row')
        for result in results:
            my_parser.insert_listing(result)      
            

a3b7aedc0c'), 'data_id': '7279274187', 'created_datetime': datetime.datetime(2021, 2, 22, 1, 51, 52, 354000), 'listing_title': '$500 MOVE IN MARCH 1st- Air conditioning, Vertical blinds, Carpet', 'listing_price': '$1,555', 'listing_url': 'https://sandiego.craigslist.org/nsd/apa/d/poway-500-move-in-march-1st-air/7279274187.html', 'listing_date': '2021-02-21 14:26', 'listing_addrcountry': 'US', 'listing_address': '13326 Community Road near Poway rd', 'listing_addrlocality': 'Poway', 'listing_addrregion': 'CA', 'listing_addrstreet': '13326 Community Road', 'listing_addrzip': '92074', 'listing_attributes': ['apartment', 'laundry on site', 'no smoking', 'off-street parking', 'rent period: monthly'], 'listing_availability': 'available mar 6', 'listing_bath': 1, 'listing_bed': '1', 'listing_bedbath': '1BR / 1Ba', 'listing_latitude': '32.957950', 'listing_longitude': '-117.040656', 'listing_petsallowed': '', 'listing_smokingallowed': 'False', 'listing_type': 'Apartment', 'listing_sqft': '600ft

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# iterate through the list of URLs, return each region listing and then run through the pages
if not config.test:
    for url in urls:
        browser.visit(url)

        # Iterate through all pages
        for x in range(1):
            # HTML object
            html_listings = browser.html
            # Parse HTML with Beautiful Soup
            soup_listings = BeautifulSoup(html_listings, 'html.parser')
            # Retrieve all elements that contain book information
            soup_listings = BeautifulSoup(html_listings)

            results = soup_listings.find_all('li', class_='result-row')

            # Loop through returned results
            for result in results:
                try:
                    my_parser.insert_listing(result)                                                                  
                except Exception as e:
                    print(e)        
            try:
                browser.links.find_by_partial_text('next').click()          
            except:
                print("Scraping Complete")
                break